Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [47]:
from random import choices, randint, choice, random
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass
from copy import copy

from pprint import pprint

import numpy as np

import lab9_lib

In [48]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .50

PROBLEM_SIZE = 100
NUM_SETS = 50

In [49]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=NUM_SETS)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00111010011110101111010001001011110011100011001001: 54.00%
00110010111011001000100011010111100100101100011101: 50.00%
00111000001111001010101001001011101100000000001010: 40.00%
00000100111111100011010111000100001110111000001101: 48.00%
01101101101011100101101111010000010111000101001101: 54.00%
11001100010110101111101110010010011010110010111000: 54.00%
01011100110100111010101001110010101001100100011110: 52.00%
00010110001001100001010100111010101001010001110100: 42.00%
00100100000111111100011100101011010111110000101101: 52.00%
01001101111011101000001010100110000000010101110000: 42.00%
10


In [50]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

In [51]:
population = [
    Individual(
        genotype=[choice((0, 1)) for _ in range(NUM_SETS)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

population.sort(key=lambda i: i.fitness, reverse=True)
for ind in population:
    print(f"{''.join(str(g) for g in ind.genotype)}: {ind.fitness:.2%}")
print(fitness.calls)

00011011101110111110000111100100111101011111111110: 66.00%
11011011010001001110110101111011011000011111110111: 64.00%
10010111100111110111100111010001111111100011001100: 62.00%
01111111011000101010011110111100100111100111100010: 60.00%
01101001111111111101011100101101011011000101100001: 60.00%
01011010100000100111111011001011100011111001111111: 60.00%
10111000010110011111010111011110101010101100111100: 60.00%
10010100111010000011011001011110111111001011010111: 58.00%
10101111101111000011111100101010110111000110011000: 58.00%
10010101001001110101101111000001100111111100011111: 58.00%
00111100010000001011011101111100111111111010110010: 58.00%
11100100000011011001011110111011111000111101101010: 58.00%
10011011101101000101111011011011010100011100101101: 58.00%
11011010111011010010110001111101011100001101011010: 58.00%
01100110111111010011000011111011100011101100011100: 58.00%
01001111100001101110101111011001011110110100100101: 58.00%
10111111110111011000000010101110001111000101010101: 56.0

In [52]:
for generation in range(1000):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print(f"best fitness: {population[0].fitness:.2%}   | generation n° {generation}")

best fitness: 66.00%   | generation n° 0
best fitness: 66.00%   | generation n° 1
best fitness: 66.00%   | generation n° 2
best fitness: 68.00%   | generation n° 3
best fitness: 70.00%   | generation n° 4
best fitness: 74.00%   | generation n° 5
best fitness: 74.00%   | generation n° 6
best fitness: 74.00%   | generation n° 7
best fitness: 74.00%   | generation n° 8
best fitness: 74.00%   | generation n° 9
best fitness: 74.00%   | generation n° 10
best fitness: 74.00%   | generation n° 11
best fitness: 74.00%   | generation n° 12
best fitness: 74.00%   | generation n° 13
best fitness: 74.00%   | generation n° 14
best fitness: 74.00%   | generation n° 15
best fitness: 74.00%   | generation n° 16
best fitness: 74.00%   | generation n° 17
best fitness: 76.00%   | generation n° 18
best fitness: 76.00%   | generation n° 19
best fitness: 76.00%   | generation n° 20
best fitness: 76.00%   | generation n° 21
best fitness: 76.00%   | generation n° 22
best fitness: 76.00%   | generation n° 23
be